<a href="https://colab.research.google.com/github/nekoniii3/Colabo_Samples/blob/main/023_DallE3_Vision/GPT_Function_Calling_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **準備：OpenAIパッケージのインストール**

In [ ]:
!pip install openai

## **準備：関数情報作成（アシスタント用）**

In [ ]:
# 関数情報
func_Dall_E3 = {
        "type": "function",
        "function": {
            "name": "request_DallE3",
            "description": "画像生成AI「dall-e-3」で指定のPromptから画像を作る。",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {"type": "string", "description": "画像を作るためのPrompt"},
                },
                "required": ["prompt"]
            }
        }
    }

func_Vision = {
        "type": "function",
        "function": {
            "name": "request_Vision",
            "description": "画像解析技術「Vision」により、指定の画像に関する質問に回答する。",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {"type": "string", "description": "画像に対する質問内容（Prompt）"},
                },
                "required": ["prompt"]
            }
        }
    }

# **準備：関数定義**

In [ ]:
from PIL import Image
from io import BytesIO
from openai import (
    OpenAI, AuthenticationError, NotFoundError, BadRequestError
)
import base64


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def request_DallE3(client, prompt, out_image_path):

    err_msg = ""

    # 今回はこれらの設定で固定
    size = "1024x1024"
    quality = "standard"

    try:

        response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size=size,
        quality=quality,
        n=1,
        response_format="b64_json"
        )

        # データを受け取りデコード
        image_data_json = response.data[0].b64_json
        image_data = base64.b64decode(image_data_json)

        # 画像として扱えるように保存
        image_stream = BytesIO(image_data)
        image = Image.open(image_stream)
        image.save(out_image_path)

    except BadRequestError as e:
        print(e)
        out_image_path = ""
        err_msg = "リクエストエラーです。著作権侵害などプロンプトを確認して下さい。"
    except Exception as e:
        print(e)
        out_image_path = ""
        err_msg = "その他のエラーが発生しました。"

    finally:

        # 結果をJSONで返す
        dalle3_result = {
            "image_path" : out_image_path,
            "error_message" : err_msg
        }
        return json.dumps(dalle3_result)


def request_Vision(client, prompt, image_path):

    response_text = ""
    err_msg = ""

    # 今回はこれらの設定で固定
    detail = "low"
    max_tokens = 300

    if image_path == "":

        # 画像がない時はエラーとして返す
        vision_result = {"answer" : "", "error_message" : "画像をセットして下さい。"}
        return json.dumps(vision_result)

    try:

        # 画像をbase64に変換
        image = encode_image(image_path)

        # メッセージの作成
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{image}",
                            "detail": detail,
                        }
                    },
                ],
            }
        ]

        # gpt-4-visionに問い合わせて回答を表示
        response = client.chat.completions.create(
            model="gpt-4-vision-preview",   # Visionはこのモデル指定
            messages=messages,
            max_tokens=max_tokens,
        )

        response_text = response.choices[0].message.content

        print(response_text)

    except BadRequestError as e:
        print(e)
        err_msg = "リクエストエラーです。画像がポリシー違反でないか確認して下さい。"
    except Exception as e:
        print(e)
        err_msg = "その他のエラーが発生しました。"

    finally:

        # 結果をJSONで返す
        vision_result = {
            "answer" : response_text,
            "error_message" : err_msg
        }
        return json.dumps(vision_result)

## **①OpenAI接続**

In [ ]:
from openai import OpenAI
import os
import json

# APKIキーのセットとクライアント接続
os.environ["OPENAI_API_KEY"] = 'sk-***************************************''

client = OpenAI()

## **②アシスタント作成 ※作成は1度で大丈夫です**

In [ ]:
# アシスタントの作成
assistant = client.beta.assistants.create(
    name="GPT_Illustrator",
    instructions="あなたはイラストレーターです。提供されている関数を使用して画像を作ったり、画像を解析したりします。",
    model="gpt-3.5-turbo-1106",
    tools=[func_Vision, func_Dall_E3]
)

assistant_id = assistant.id
print(assistant_id)

### **※アシスタントが既にあればIDをセット**

In [ ]:
assistant_id = "asst_XXXXXXXXXXXXXXXXXXXXXX"

## **③メッセージの作成**

In [ ]:
# スレッドの作成
thread = client.beta.threads.create()

In [ ]:
# prompt = "青い目のシャム猫の画像を作ってください。"
# prompt = "次の画像が何を表しているか教えてください。"
prompt = "「1980s anime girl with straight bob-cut in school uniform, roughly drawn drawing」で画像を作ってください。"

# メッセージの作成
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

In [ ]:
# RUNスタート（アシスタントへ問い合わせ）
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant_id
)

In [ ]:
# スレッドの実行状態を受け取る
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

# 状態を確認
print(run.status)

## **④関数の結果の提出**

In [ ]:
if run.status == "requires_action":   # 関数の結果の提出待ち

    # ここに入力画像のファイルパス設定(Visionの場合のみ)
    image_path = "./image.jpg"

    # 出力画像のファイルパスを適当に設定(dall-e-3の場合のみ)
    out_image_path = "out_image.jpg"

    tool_id = run.required_action.submit_tool_outputs.tool_calls[0].id
    func_name = run.required_action.submit_tool_outputs.tool_calls[0].function.name
    func_args = json.loads(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments)

    print("id:", tool_id)
    print("name:", func_name)
    print("arguments:", func_args)

    if func_name == "request_DallE3":

        func_output = request_DallE3(
            client,
            func_args["prompt"],
            out_image_path   # 出力パス
        )

    elif func_name == "request_Vision":

        func_output = request_Vision(
            client,
            func_args["prompt"],
            image_path
        )



In [ ]:
# 関数の出力を提出
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_id,
            "output": func_output,
        }
    ]
)

## **⑤再度メッセージを受け取る**

In [ ]:
# スレッドの実行状態を受け取る
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

# 状態を確認
print(run.status)

In [ ]:
# スレッドのメッセージリストの確認
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)
for message in messages:
    print(message.role, ":", message.content[0].text.value)

# **画像の確認（Dall-E-3のみ）**

In [ ]:
from IPython.display import display

out_image = Image.open(out_image_path)

display(out_image)

# **（テスト用の関数）**

In [ ]:
def request_DallE3(client, prompt, out_image_path):
    """ DallE3を呼び出す """

    err_msg = ""

    dalle3_result = {
        "image_path" : out_image_path,
        "error_message" : err_msg
    }

    return json.dumps(dalle3_result)


def request_Vision(client, prompt, image_path):
    """ GPT4 Visionを呼び出す """

    response_text = "この画像は驚いた表情をしている人物を写した写真です。"
    err_msg = ""

    vision_result = {
        "answer" : response_text,
        "error_message" : err_msg
    }

    return json.dumps(vision_result)